## 20170628 Tensorflow上午實作部分程式碼

In [2]:
# Packages
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from numpy import genfromtxt

In [3]:
# Read csv
def read_dataset(path_to_file):
    return genfromtxt(path_to_file, delimiter=delimiter, skip_header=1)

def read_boston_data():
    data = read_dataset(path_to_file)
    features = data[:, :-1]
    prices = data[:, -1]
    return features, prices

# main
# path_to_file 放自己檔案位置
path_to_file='/Users/chia/Downloads/boston.csv'
delimiter=','
features, price=read_boston_data()

In [4]:
# Feature Normalization 統計標準化，使變數服從常態分配
def feature_normalize(dataset):
    mu=np.mean(dataset,axis=0)
    sigma=np.std(dataset,axis=0)
    return(dataset-mu)/sigma

# main
normalized_features = feature_normalize(features)

In [48]:
# Adding Constant/Bias 模型偏差
def append_bias_reshape(features,price):
    n_samples, n_dim = features.shape
    expended_features=np.c_[np.ones(n_samples),features]
    return expended_features, price.reshape(n_samples,1)

# main
f,p=append_bias_reshape(normalized_features,price)

# 這一段只是在整個資料矩陣最左邊加入全為'1'的欄，維度是(506, 14)，叫做f。
# 可以用以下方式觀察
# print(f)
# print(f[0,])
# p則是模型的Ｙ值，維度為(506, 1)。

In [6]:
# Train test split
random_indices = np.random.rand(len(f)) < 0.8

x_train = f[random_indices]
y_train = p[random_indices]
x_test = f[~random_indices]
y_test = p[~random_indices]

# 這段是產生隨機的模型訓練資料及驗測資料。

In [67]:
# Parameters
n_dim=f.shape[1]
learning_rate=0.01
training_epochs=1000
display_step=50
cost_history=np.empty(shape=[1],dtype=float)
seed=2017

# Constructing graph
x = tf.placeholder(tf.float32, shape=[None, n_dim])
y = tf.placeholder(tf.float32, shape=[None, 1])

In [71]:
# Set model weights
w = tf.Variable(tf.random_normal([n_dim, 1], seed=seed), name="weight")

# Define cost function
y_ = tf.matmul(x, w)
cost = tf.reduce_mean(tf.square(y_ - y))
training_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# 這段是建立一個模型y_，他跟真實模型會有一個誤差，目的是把誤差減到最小，即y_-y的值。

In [75]:
# Session & Initialization
sess=tf.Session()
init=tf.global_variables_initializer()
sess.run(init)

In [79]:
# Training
for epoch in range(training_epochs):
    sess.run(training_step,
    feed_dict={x:x_train,y:y_train})
    cost_history=np.append(cost_history,
                          sess.run(cost,feed_dict={x:x_train,y:y_train}))

## 下午實作部分

In [38]:
from tensorflow.examples.tutorials.mnist import input_data
# data
mnist=input_data.read_data_sets("MNIST/",one_hot=True)

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Model parameters
learning_rate=0.01
epochs = 20
batch_size=100
nb_batch=int(mnist.train.num_examples/batch_size)
display_step=1
cost_history=list()

Extracting MNIST/train-images-idx3-ubyte.gz
Extracting MNIST/train-labels-idx1-ubyte.gz
Extracting MNIST/t10k-images-idx3-ubyte.gz
Extracting MNIST/t10k-labels-idx1-ubyte.gz


In [39]:
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])
W=tf.Variable(tf.zeros([784,10]))
b=tf.Variable(tf.zeros([10]))
y=tf.add(tf.matmul(x,W),b)

In [40]:
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_,logits=y))
train_step=tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
sess=tf.InteractiveSession()
tf.global_variables_initializer().run()

In [41]:
for epoch in range(epochs):
    avg_cost=0.
    for _ in range(nb_batch):
        batch_xs, batch_ys=mnist.train.next_batch(batch_size)
        _,c=sess.run([train_step,cost],
                    feed_dict={x:batch_xs,y_:batch_ys})
        avg_cost+=c/nb_batch
        
    cost_history.append(avg_cost)
    if(epoch+1)%display_step==0:
        print("Epoch:{:04d} Cost={:.9f}".format(epoch+1,avg_cost))

Epoch:0001 Cost=1.183907932
Epoch:0002 Cost=0.665371387
Epoch:0003 Cost=0.552838534
Epoch:0004 Cost=0.498707824
Epoch:0005 Cost=0.465535456
Epoch:0006 Cost=0.442627445
Epoch:0007 Cost=0.425560299
Epoch:0008 Cost=0.412233437
Epoch:0009 Cost=0.401436647
Epoch:0010 Cost=0.392377033
Epoch:0011 Cost=0.384775886
Epoch:0012 Cost=0.378189199
Epoch:0013 Cost=0.372416119
Epoch:0014 Cost=0.367337827
Epoch:0015 Cost=0.362717281
Epoch:0016 Cost=0.358619152
Epoch:0017 Cost=0.354898577
Epoch:0018 Cost=0.351475001
Epoch:0019 Cost=0.348347098
Epoch:0020 Cost=0.345429354


In [43]:
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x:mnist.test.images,
                                   y_:mnist.test.labels}))

0.911


In [84]:
from __future__ import print_function 
import tensorflow as tf 
import matplotlib.pyplot as plt 
import numpy as np 
from tensorflow.examples.tutorials.mnist import input_data 
mnist = input_data.read_data_sets("MNIST/", one_hot=True)
                                  
learning_rate = 0.01 
epochs = 20
batch_size = 100 
nb_batch = int(mnist.train.num_examples/batch_size) 
display_step = 1
cost_history = list() 


Extracting MNIST/train-images-idx3-ubyte.gz
Extracting MNIST/train-labels-idx1-ubyte.gz
Extracting MNIST/t10k-images-idx3-ubyte.gz
Extracting MNIST/t10k-labels-idx1-ubyte.gz


In [85]:
n_hidden_1 = 256 
n_hidden_2 = 64 
n_input_dim = 784 
n_class = 10 

In [86]:
# Store weights and bias definition 
weights = { 
    'h1': tf.Variable(tf.random_normal([n_input_dim, n_hidden_1])), 
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])), 
    'h3': tf.Variable(tf.random_normal([n_hidden_2, n_class])) 
} 
bias = { 
    'b1': tf.Variable(tf.random_normal([n_hidden_1])), 
    'b2': tf.Variable(tf.random_normal([n_hidden_2])), 
    'b3': tf.Variable(tf.random_normal([n_class])) 
}

In [87]:
# Create model
def multilayer_perceptron(x, weights, bias): 
    # Hidden Layer 1 - ReLu activation 
    layer_1 = tf.add(tf.matmul(x, weights['h1']), bias['b1']) 
    layer_1 = tf.nn.relu(layer_1) 
    
    # Hidden Layer 2 - ReLu activation 
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), bias['b2']) 
    layer_2 = tf.nn.relu(layer_2) 
    
    # Output Layer Linear activation 
    layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), bias['b3']) 
    return layer_3 


In [88]:
#constructing graph input (input feature normatized) 
x = tf.placeholder(tf.float32, [None, n_input_dim]) # MIST input 28x28 -> 784 
y_ = tf.placeholder(tf.float32, [None, n_class]) # 10 digits -> 10 cLasses 
y = multilayer_perceptron(x, weights, bias) 
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, 
                                                              logits=y)) 
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost) 


In [89]:
# Creo - 
sess = tf.InteractiveSession() 
# Initialize the variabtef 
tf.global_variables_initializer().run() 


In [90]:
for epoch in range(epochs):
    avg_cost=0.
    for _ in range(nb_batch):
        batch_xs, batch_ys=mnist.train.next_batch(batch_size)
        _,c=sess.run([train_step,cost],
                    feed_dict={x:batch_xs,y_:batch_ys})
        avg_cost+=c/nb_batch
        
    cost_history.append(avg_cost)
    # display log
    if(epoch+1)%display_step==0:
        print("Epoch:{:04d} Cost={:.9f}".format(epoch+1,avg_cost))
        
print('Finished Optimization!')

Epoch:0001 Cost=22.025981406
Epoch:0002 Cost=2.054608068
Epoch:0003 Cost=1.597285672
Epoch:0004 Cost=1.407760536
Epoch:0005 Cost=1.305904414
Epoch:0006 Cost=1.236106888
Epoch:0007 Cost=1.173384563
Epoch:0008 Cost=1.129876092
Epoch:0009 Cost=1.091303134
Epoch:0010 Cost=1.044371406
Epoch:0011 Cost=1.012794874
Epoch:0012 Cost=0.977672402
Epoch:0013 Cost=0.947622979
Epoch:0014 Cost=0.933352177
Epoch:0015 Cost=0.897951126
Epoch:0016 Cost=0.880312209
Epoch:0017 Cost=0.858760628
Epoch:0018 Cost=0.835389947
Epoch:0019 Cost=0.819633080
Epoch:0020 Cost=0.793951937
Finished Optimization!


In [91]:
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x:mnist.test.images,
                                   y_:mnist.test.labels}))

0.7849


In [114]:
# Create modeL
def nultilayer_perceptron(x, weights, bias):
    # Hidden Layer 1 - ReLu activation
    layer_1 = tf.add(tf.natnu1(x, weights['w1']), bias['b1'])
    layer_1 = tf.nn.re1u(layer_1)
    tf.summary.histogram("re1u1", layer_1)
    # Hidden Layer 2 - ReLu activation
    layer_2 = tf.add(tf.natnu1(layer_1, weights['w2']), biasl['b2'])
    layer_2 = tf.nn.re1u(layer_2)
    tf.summary.histogram("re1u2", layer_2)
    # Output Layer - Linear activation
    layer_3 = tf.add(tf.natnu1(layer_2, ueights['w3']), biasl['b3'])
    return layer_3

In [113]:
from __future__ import print_function

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST/", one_hot=True)


# Model parameters
learning_rate = 0.01
epochs = 50
batch_size = 100
nb_batch = int(mnist.train.num_examples/batch_size)
display_step = 1
logs_path = './logs/'


# Network parameters
n_hidden_1 = 256
n_hidden_2 = 64
n_input_dim = 784
n_class = 10


# Constructing graph input (input feature normalized)
x = tf.placeholder(tf.float32, [None, n_input_dim])  # MNIST input 28x28 -> 784
y = tf.placeholder(tf.float32, [None, n_class])  # 10 digits -> 10 classes

# Xavier uniform initializer


# Create model
def multilayer_perceptron(x, weights, bias):
    # Hidden layer 1 - Relu activation
    layer_1 = tf.add(tf.matmul(x, weights['w1']), bias['b1'])
    layer_1 = tf.nn.relu(layer_1)
    tf.summary.histogram("relu1", layer_1)

    # Hidden layer 2 - Relu activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), bias['b2'])
    layer_2 = tf.nn.relu(layer_2)
    tf.summary.histogram("relu2", layer_2)

    # Output layer - Linear activation
    layer_3 = tf.add(tf.matmul(layer_2, weights['w3']), bias['b3'])
    return layer_3


# tf tensorboard summary page
with tf.name_scope('Weights'):
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input_dim, n_hidden_1]),
                          name='w1'),
        'w2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]),
                          name='w2'),
        'w3': tf.Variable(tf.random_normal([n_hidden_2, n_class]),
                          name='w3')
    }


with tf.name_scope('Bias'):
    bias = {
        'b1': tf.Variable(tf.random_normal([n_hidden_1]),
                          name='b1'),
        'b2': tf.Variable(tf.random_normal([n_hidden_2]),
                          name='b2'),
        'b3': tf.Variable(tf.random_normal([n_class]),
                          name='b3')
    }

with tf.name_scope('Model'):
    # Build model
    pred = multilayer_perceptron(x, weights, bias)

with tf.name_scope('Loss'):
    # Softmax Cross entropy (cost function)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,
                                                                  labels=y))

with tf.name_scope('SGD'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    grads = tf.gradients(loss, tf.trainable_variables())
    grads = list(zip(grads, tf.trainable_variables()))
    apply_grads = optimizer.apply_gradients(grads_and_vars=grads)

with tf.name_scope('Accuracy'):
    # Accuracy
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))


# Create a summary to monitor cost tensor
tf.summary.scalar("loss", loss)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("accuracy", acc)
# Create summaries to visualize weights
for var in tf.trainable_variables():
    tf.summary.histogram(var.name, var)
# Merge all summaries into a single op
merged_summary = tf.summary.merge_all()


# Create session
sess = tf.InteractiveSession()

# Initialize the variables
tf.global_variables_initializer().run()

# op to write logs to Tensorboard
summary_writer = tf.summary.FileWriter(logs_path,
                                       graph=tf.get_default_graph())

# Start training
for epoch in range(epochs):
    avg_cost = 0.
    for i in range(nb_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        _, c, summary = sess.run([apply_grads, loss, merged_summary],
                        feed_dict={x: batch_xs, y: batch_ys})
        # Write logs at every iteration
        summary_writer.add_summary(summary, epoch * nb_batch + i)
        avg_cost += c/nb_batch
    # display log
    if (epoch+1) % display_step == 0:
        print("Epoch: {:04d}   Cost= {:.9f}".format(epoch+1, avg_cost))

print ('Finished Optimization!')

# Test model
# Calculate accuracy
print ("Accuracy:", acc.eval({x: mnist.test.images, y: mnist.test.labels}))

print ("Run the command line:\n" \
       "--> tensorboard --logdir=./logs/\n" \
       "Then open http://0.0.0.0:6006/ into your web browser")


Extracting MNIST/train-images-idx3-ubyte.gz
Extracting MNIST/train-labels-idx1-ubyte.gz
Extracting MNIST/t10k-images-idx3-ubyte.gz
Extracting MNIST/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Summary name Variable:0 is illegal; using Variable_0 instead.
INFO:tensorflow:Summary name Variable_1:0 is illegal; using Variable_1_0 instead.
INFO:tensorflow:Summary name Variable_2:0 is illegal; using Variable_2_0 instead.
INFO:tensorflow:Summary name Variable_3:0 is illegal; using Variable_3_0 instead.
INFO:tensorflow:Summary name Variable_4:0 is illegal; using Variable_4_0 instead.
INFO:tensorflow:Summary name Variable_5:0 is illegal; using Variable_5_0 instead.
INFO:tensorflow:Summary name Variable_6:0 is illegal; using Variable_6_0 instead.
INFO:tensorflow:Summary name Variable_7:0 is illegal; using Variable_7_0 instead.
INFO:tensorflow:Summary name Variable_8:0 is illegal; using Variable_8_0 instead.
INFO:tensorflow:Summary name Variable_9:0 is illegal; using Variable_9_0 instead.
INFO:tenso

InvalidArgumentError: Shape [-1,784] has negative dimensions
	 [[Node: Placeholder_24 = Placeholder[dtype=DT_FLOAT, shape=[?,784], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'Placeholder_24', defined at:
  File "/Users/chia/anaconda/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/chia/anaconda/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/chia/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/chia/anaconda/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/chia/anaconda/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/Users/chia/anaconda/lib/python3.6/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "/Use

In [ ]:
tensorboard --logdir='/Users/chia/logs'